In [ ]:
!pip install easyocr streamlit pillow


In [ ]:
%%writefile app.py


import streamlit as st
from PIL import Image
import cv2
import easyocr
import sqlite3
import re
import numpy as np

# Function to split and extract text using EasyOCR
def split_and_extract_text(image):
    height, width, _ = image.shape

    half_width = width // 2
    left_half = image[:, :half_width]
    right_half = image[:, half_width:]

    left_text = extract_text(left_half)
    right_text = extract_text(right_half)

    company_name = get_company_name(left_text, right_text)

    combined_text = left_text + ' ' + right_text

    return company_name, combined_text



def extract_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['en'])
    results = reader.readtext(gray)
    extracted_text = ' '.join([result[1] if isinstance(result[1], str) else result[1].decode('utf-8') for result in results])
    return extracted_text

def get_company_name(left_text, right_text):
    if len(left_text.split()) == 2:
        return left_text
    elif len(right_text.split()) == 2:
        return right_text
    else:
        return 'Family Restaurant'

# Function to process extracted information
def get_data(res, company_name):
    data = {
        "company_name": company_name,
        "card_holder_name": "",
        "designation": "",
        "mobile_number": "",
        "email": "",
        "website_url": "",
        "area": "",
        "city": "",
        "state": "",
        "pin_code": "",
    }

    if res:
        for ind, i in enumerate(res):
            # To get CARD HOLDER NAME
            if ind == 0:
                data["card_holder_name"] = i

            # To get DESIGNATION
            elif ind == 1:
                data["designation"] = i

            # To get MOBILE NUMBER
            elif re.search(r'mobile|\+\d{2}', str(i), re.IGNORECASE) or re.search(r'^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$', str(i), re.IGNORECASE):
                data["mobile_number"] = i

            # To get EMAIL ID
            elif re.search(r'\S+@\S+', str(i)):
                data["email"] = i

            # To get WEBSITE_URL
            elif re.search(r'www', str(i), re.IGNORECASE):
                data["website_url"] = i

            # To get AREA, CITY, and STATE
            elif re.search(r'\d+ [a-zA-Z]+ St', str(i)) or re.search(r'[a-zA-Z]+ St,,', str(i)) or re.search(r'[a-zA-Z]+ St.,', str(i)):
                address_parts = str(i).split(',')
                if len(address_parts) >= 3:
                    data["area"] = address_parts[0].strip()
                    data["city"] = address_parts[1].strip()
                    data["state"] = address_parts[2].strip()

            # To get PINCODE
            elif len(str(i)) >= 5 and str(i).isdigit():
                data["pin_code"] = i

            if data["state"] is None or data["state"] == "":
                    data["state"] = 'TamilNadu'

            if data["pin_code"] is None or data["pin_code"] == "":
                    data["pin_code"] = '6004513'




    return data


# Set up easyOCR reader
reader = easyocr.Reader(['en'])

# Create SQLite database and table
conn = sqlite3.connect('business_cards.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS business_cards (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        company_name TEXT,
        card_holder_name TEXT,
        designation TEXT,
        mobile_number TEXT,
        email TEXT,
        website_url TEXT,
        area TEXT,
        city TEXT,
        state TEXT,
        pin_code TEXT,
        image_path TEXT
    )
''')
conn.commit()

# Streamlit UI
st.title("Business Card Information Extractor")

# Upload image
uploaded_image = st.file_uploader("Upload Business Card Image", type=["jpg", "jpeg", "png"])

if uploaded_image:
    pil_image = Image.open(uploaded_image)
    st.image(pil_image, caption="Uploaded Business Card", use_column_width=True)
    np_image = np.array(pil_image)

    company_name, combined_text = split_and_extract_text(np_image)
    extracted_info = reader.readtext(np_image)
    data = get_data([item[1] for item in extracted_info], company_name)

    st.subheader("Extracted Information:")
    st.write(data)

    if st.button("Save to Database"):
        cursor.execute('''
            INSERT INTO business_cards (
                company_name, card_holder_name, designation, mobile_number, email,
                website_url, area, city, state, pin_code, image_path
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            company_name, data["card_holder_name"], data["designation"],
            data["mobile_number"], data["email"], data["website_url"],
            data["area"], data["city"], data["state"], data["pin_code"],
            uploaded_image.name
        ))
        conn.commit()
        st.success("Information saved to the database.")


# ... (rest of the code remains unchanged)
# Read from the database
st.subheader("Read from Database")
data = cursor.execute('SELECT * FROM business_cards').fetchall()
for entry in data:
    st.write(entry)

# Update and Delete from the database
selected_entry = st.selectbox("Select an entry to update or delete:", [entry[0] for entry in data])

if selected_entry:
    action = st.radio("Choose action:", ["Update", "Delete"])

    if action == "Update":
        st.subheader("Update Entry")

        # Get updated information from the user
        updated_info_dict = {
            "company_name": st.text_input("Company Name", value=data[selected_entry - 1][1]),
            "card_holder_name": st.text_input("Card Holder Name", value=data[selected_entry - 1][2]),
            "designation": st.text_input("Designation", value=data[selected_entry - 1][3]),
            "mobile_number": st.text_input("Mobile Number", value=data[selected_entry - 1][4]),
            "email": st.text_input("Email", value=data[selected_entry - 1][5]),
            "website_url": st.text_input("Website URL", value=data[selected_entry - 1][6]),
            "area": st.text_input("Area", value=data[selected_entry - 1][7]),
            "city": st.text_input("City", value=data[selected_entry - 1][8]),
            "state": st.text_input("State", value=data[selected_entry - 1][9]),
            "pin_code": st.text_input("Pin Code", value=data[selected_entry - 1][10]),
        }

        if st.button("Update"):
            # Update the entry in the database
            cursor.execute('''
                UPDATE business_cards
                SET company_name=?, card_holder_name=?, designation=?, mobile_number=?, email=?,
                website_url=?, area=?, city=?, state=?, pin_code=?
                WHERE id=?
            ''', (
                updated_info_dict["company_name"], updated_info_dict["card_holder_name"],
                updated_info_dict["designation"], updated_info_dict["mobile_number"],
                updated_info_dict["email"], updated_info_dict["website_url"],
                updated_info_dict["area"], updated_info_dict["city"],
                updated_info_dict["state"], updated_info_dict["pin_code"],
                selected_entry
            ))
            conn.commit()
            st.success("Entry updated successfully.")

    elif action == "Delete":
        st.subheader("Delete Entry")

        if st.button("Delete"):
            # Delete the entry from the database
            cursor.execute('DELETE FROM business_cards WHERE id=?', (selected_entry,))
            conn.commit()
            st.success("Entry deleted successfully.")

# Close the database connection
conn.close()


In [3]:
!wget -q -O - ipv4.icanhazip.com

34.141.155.99


In [4]:
! npm install -g localtunnel


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.397s


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.155.99:8501

npx: installed 22 in 5.425s
your url is: https://fancy-sheep-grin.loca.lt
/content/app.py:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if data["state"] is '':
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Progress: |██████████████████████████████████████████████████| 100.0% CompleteDownloading recognition model, please wait. This may take several minutes depending upon your network connection.
Progress: |██████████████████████████████████████████████████| 100.0% CompleteNeither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
[ WARN:0@64.513] global loadsave.cpp:248 findDecoder imread_('3.png'): can't open/read file: check file path/integrity
2024-01-23 14:32:24.343 Uncaught app exception
Traceback (most recent call last):
  File "/u